In [2]:
%%capture
# Instalar dependências no ambiente virtual do Jupyter
%pip install psycopg2-binary
%pip install pandas
%pip install panel

In [1]:
# Definindo as variáveis com informações sobre o db

dbname   = 'final' # nome do db
user     = 'sgbd'  # nome do usuario que tem acesso ao db  
password = '1234'  # senha do usuario

In [ ]:

# Importando as dependências e conectando ao banco de dados. Também uso warnings para ignorar avisos de compatibilidade entre o pandas e o psycopg2.

import psycopg2 as psy
import pandas as pd
import panel as pn
import warnings


warnings.filterwarnings('ignore')

pn.extension()

connection = psy.connect(dbname=dbname, user=user, password=password)
cursor = connection.cursor()
print("Conexão com banco de dados realizada com sucesso")# Importando as dependências e conectando ao banco de dados. Também uso warnings para ignorar avisos de compatibilidade entre o pandas e o psycopg2.

import psycopg2 as psy
import pandas as pd
import panel as pn
import warnings


warnings.filterwarnings('ignore')

pn.extension()

connection = psy.connect(dbname=dbname, user=user, password=password)
cursor = connection.cursor()
print("Conexão com banco de dados realizada com sucesso")

In [ ]:
# Obter toda a tabela

df = pd.read_sql('SELECT * FROM usuario', connection)
display(pn.pane.DataFrame(df, width=700))

In [ ]:
# Pegar um usuário específico a partir do nome

text_input = pn.widgets.TextInput(name="Digite um nome de usuário:")
button = pn.widgets.Button(name="Buscar", button_type="primary")
pn_df = pn.pane.DataFrame(pd.DataFrame(), width=700)

def on_click(event):
    nome = text_input.value.strip()
    pd_df = pd.read_sql(f"SELECT * FROM usuario WHERE nome ILIKE %s", connection, params=(f'%{nome}%',))
    pn_df.object = pd_df  # Atualiza o conteúdo em vez de recriar o objeto

button.on_click(on_click)

display(text_input, button, pn_df)# Pegar um usuário específico a partir do nome

text_input = pn.widgets.TextInput(name="Digite um nome de usuário:")
button = pn.widgets.Button(name="Buscar", button_type="primary")
pn_df = pn.pane.DataFrame(pd.DataFrame(), width=700)

def on_click(event):
    nome = text_input.value.strip()
    pd_df = pd.read_sql(f"SELECT * FROM usuario WHERE nome ILIKE %s", connection, params=(f'%{nome}%',))
    pn_df.object = pd_df  # Atualiza o conteúdo em vez de recriar o objeto

button.on_click(on_click)

display(text_input, button, pn_df)

In [ ]:
# Remover um usuário a partir do ID

text_input = pn.widgets.TextInput(name="Digite o ID do usuário a ser deletado:")
button = pn.widgets.Button(name="Buscar", button_type="primary")
pn_df = pn.pane.DataFrame(pd.DataFrame(), width=700)

def on_click(event):
    global pn_df
    id_usuario = text_input.value.strip()
    pd_df = pd.read_sql("SELECT * FROM usuario WHERE id_usuario = %s", connection, params=(id_usuario,))
    try:
        cursor.execute("""
                DELETE FROM pagamento
                WHERE id_doacao IN (
                    SELECT d.id_doacao
                    FROM doacao d
                    INNER JOIN instituicao i ON d.id_instituicao = i.id_instituicao
                    WHERE i.id_usuario = %s
                );
            """, (id_usuario,))

        # 2. Deletar pagamentos associados às doações em campanhas das instituições do usuário
        cursor.execute("""
            DELETE FROM pagamento
            WHERE id_doacao IN (
                SELECT d.id_doacao
                FROM doacao d
                INNER JOIN campanha c ON d.id_campanha = c.id_campanha
                INNER JOIN instituicao i ON c.id_instituicao = i.id_instituicao
                WHERE i.id_usuario = %s
            );
        """, (id_usuario,))

        # 3. Deletar doações feitas às instituições do usuário
        cursor.execute("""
            DELETE FROM doacao
            WHERE id_instituicao IN (
                SELECT id_instituicao FROM instituicao WHERE id_usuario = %s
            );
        """, (id_usuario,))

        # 4. Deletar doações feitas para campanhas gerenciadas pelo usuário
        cursor.execute("""
            DELETE FROM doacao
            WHERE id_campanha IN (
                SELECT c.id_campanha
                FROM campanha c
                INNER JOIN instituicao i ON c.id_instituicao = i.id_instituicao
                WHERE i.id_usuario = %s
            );
        """, (id_usuario,))

        # 5. Deletar campanhas associadas às instituições do usuário
        cursor.execute("""
            DELETE FROM campanha
            WHERE id_instituicao IN (
                SELECT id_instituicao FROM instituicao WHERE id_usuario = %s
            );
        """, (id_usuario,))

        # 6. Deletar instituições associadas ao usuário
        cursor.execute("""
            DELETE FROM instituicao WHERE id_usuario = %s;
        """, (id_usuario,))

        # 7. Finalmente, deletar o usuário
        cursor.execute("""
            DELETE FROM usuario WHERE id_usuario = %s;
        """, (id_usuario,))

        cursor.execute("DELETE FROM instituicao WHERE id_usuario = %s;", (id_usuario,))
        cursor.execute("DELETE FROM usuario WHERE id_usuario = %s;", (id_usuario,))
        
        pn_df.object = pd_df
    except Exception as e:
        print("Erro ao deletar usuário:", e)

button.on_click(on_click)

display(text_input, button, pn.pane.Markdown("**Deletado:**"), pn_df)


In [ ]:
# Adicionar um usuário
text_input_nome = pn.widgets.TextInput(name="Digite um nome para o usuário:")
text_input_email = pn.widgets.TextInput(name="Digite um email para o usuário:")
text_input_telefone = pn.widgets.TextInput(name="Digite um telefone:")

pn_df = pn.pane.DataFrame(pd.DataFrame(), width=700)

def formatar_telefone(telefone):
    import re
    # Remove caracteres não numéricos
    telefone = re.sub(r'\D', '', telefone)
    
    # Verifica se o telefone tem o número correto de dígitos (11 para celular no Brasil)
    if len(telefone) == 11:
        # Formata como (XX) XXXXX-XXXX
        telefone_formatado = f"({telefone[:2]}) {telefone[2:7]}-{telefone[7:]}"
    elif len(telefone) == 10:
        # Formata como (XX) XXXX-XXXX (número fixo)
        telefone_formatado = f"({telefone[:2]}) {telefone[2:6]}-{telefone[6:]}"
    else:
        # Se o número não for válido, retorna uma mensagem de erro
        telefone_formatado = "Número de telefone inválido"
    
    return telefone_formatado

def adicionar_usuario(event):
    global cursor
    nome = text_input_nome.value.strip()
    email = text_input_email.value.strip()
    telefone = text_input_telefone.value.strip()
    
    if nome and email and telefone: 
        cursor.execute("""
            INSERT INTO usuario (nome, email, telefone)
            VALUES (%s, %s, %s);
        """, (nome, email.lower(), formatar_telefone(telefone)))
        
        pd_df = pd.read_sql("SELECT * FROM usuario", connection)
        pn_df.object = pd_df
        
    else:
        print("Por favor, preencha todos os campos.")

add_button = pn.widgets.Button(name="Adicionar Usuário")
add_button.on_click(adicionar_usuario)

display(text_input_nome, text_input_email, text_input_telefone, add_button, pn_df)

In [ ]:
# Salvar ou reverter alterações

def salvar(event):
    connection.commit()
    print('Alterações salvas')

def reverter(event):
    connection.rollback()
    print('Alterações desfeitas')

salvar_button = pn.widgets.Button(name="Salvar")
rollback_button = pn.widgets.Button(name="Reverter")

salvar_button.on_click(salvar)
rollback_button.on_click(reverter)

display(salvar_button, rollback_button)